
# Bloodborne IF – Game State Controller

This notebook defines the game state controller for our Bloodborne-inspired
text-based RPG

It reads from the project dataset to build the world graph:

- Data/Locations/*.json – location id, name, description, previous/next links,
  characters (and counts), and items (and counts)
- Data/Items/*.json – item metadata (id, name, alias, description, etc.)

The controller tracks:

- Weapon upgrade level
- Current & max health
- Current location
- Objects that can be interacted with in the current location
- Inventory (stacked counts, e.g. blood vial (x2))
- Blood Echoes
- Total lamps unlocked
- Player Insight level & current location Insight level
- Characters present in the current location (with counts)
- Which locations have been visited at least once (for Insight gain)


In [1]:

from collections import defaultdict
import io
import sys
import os
import json
import zipfile

# Control where game output goes (stdout vs hidden buffer)
print_to_stdout = True
print_to_devnull = False

if print_to_stdout:
    game_io = sys.stdout
elif print_to_devnull:
    game_io = open(os.devnull, 'w')
else:
    game_io = io.StringIO()

def print_game_output(*args, **kwargs):
    """Helper used by the game engine to print text to the player."""
    kwargs['file'] = game_io
    print(*args, **kwargs)



## World Primitives: Location and Item

Location tracks:

- id (dataset id)  
- name, description  
- characters in the location (character_id -> count)  
- items + item_counts (stacks of items)  
- insight_level (area intensity)  
- has_been_visited (for compatibility / flavor)


In [ ]:

class Location:
    """Locations are the places in the game that a player can visit.
       They are constructed from the dataset's location JSON.
    """
    def __init__(self, loc_id, name, description, insight_level=0):
        self.id = loc_id
        self.name = name
        self.description = description
        self.properties = defaultdict(bool)
        self.connections = {}
        self.travel_descriptions = {}
        self.items = {}
        self.item_counts = defaultdict(int)
        self.blocks = {}
        self.has_been_visited = False
        # How "cosmic" / eldritch this location feels
        self.insight_level = insight_level
        # Characters present
        self.characters = defaultdict(int)

    def set_property(self, property_name, property_bool=True):
        self.properties[property_name] = property_bool

    def get_property(self, property_name):
        return self.properties[property_name]

    def set_insight_level(self, value: int):
        self.insight_level = value

    def get_insight_level(self) -> int:
        return self.insight_level

    def add_connection(self, direction, connected_location, travel_description=""):
        direction = direction.lower()
        self.connections[direction] = connected_location
        self.travel_descriptions[direction] = travel_description

        # Automatically add reverse connections for basic directions
        reverse = {
            'north': 'south', 'south': 'north',
            'east': 'west',   'west': 'east',
            'up': 'down',     'down': 'up',
        }
        if direction in reverse:
            rev = reverse[direction]
            if rev not in connected_location.connections:
                connected_location.connections[rev] = self
                connected_location.travel_descriptions[rev] = ""

    def add_block(self, blocked_direction, block_description, preconditions):
        """Add a puzzle/block to a direction.
        `preconditions` is a list of callables: game -> bool.
        """
        self.blocks[blocked_direction] = (block_description, preconditions)

    def is_blocked(self, direction, game):
        if direction not in self.blocks:
            return False
        block_description, preconditions = self.blocks[direction]
        if all(cond(game) for cond in preconditions):
            return False
        print_game_output(block_description)
        return True

    # ----- Items in location (stacked) -----
    def add_item(self, item, count=1):
        if item.name not in self.items:
            self.items[item.name] = item
        self.item_counts[item.name] += count

    def remove_item(self, item_name, count=1):
        """Remove up to `count` of an item; returns a canonical Item or None.
        For compatibility, we return a single `Item` instance even if more than
        one copy existed.
        """
        if self.item_counts[item_name] <= 0:
            return None
        item = self.items.get(item_name)
        if item is None:
            return None
        self.item_counts[item_name] -= count
        if self.item_counts[item_name] <= 0:
            self.item_counts[item_name] = 0
            self.items.pop(item_name, None)
        return item

    # ----- Characters in location -----
    def add_character(self, char_id, count=1):
        self.characters[char_id] += count

    def get_characters_summary(self):
        return dict(self.characters)


class Item:
    """Items are objects that can be picked up, examined, and used.
    They are constructed from the Items JSON in the dataset.
    """
    def __init__(self, item_id, name, description, properties=None):
        self.id = item_id
        self.name = name
        self.description = description
        self.properties = defaultdict(bool)
        if properties:
            for k, v in properties.items():
                self.properties[k] = v

    def set_property(self, prop, value=True):
        self.properties[prop] = value

    def get_property(self, prop):
        return self.properties[prop]



## BloodborneGame – Core State Tracking

The BloodborneGame class tracks:

- Player health & max health  
- Player weapon upgrade level  
- Player Blood Echoes  
- Player Insight  
- Current location & its insight level  
- Inventory with stacked counts  
- Characters & items in the current location  
- Lamps unlocked (by location id)  
- Visited locations (location ids) — and gives +1 Insight the first time
  you ever enter any location (including the starting one).


In [3]:

class BloodborneGame:
    def __init__(self, start_at, all_locations,
                 max_health=100, weapon_level=1,
                 starting_insight=0, max_insight=99,
                 starting_blood_echoes=0):
        # World graph
        self.curr_location = start_at
        self.all_locations = all_locations  # dict loc_id -> Location

        # Player-centric state
        self.max_health = max_health
        self.health = max_health
        self.weapon_level = weapon_level

        # Blood Echoes
        self.blood_echoes = starting_blood_echoes

        # Insight tracking
        self.max_insight = max_insight
        self.insight = starting_insight

        # Inventory: canonical item + stacked counts
        self.inventory = {}              # name -> Item
        self.inventory_counts = defaultdict(int)  # name -> count

        # Lamps: track how many unique lamps have been unlocked (by location id)
        self.unlocked_lamp_locations = set()  # location.id where lamp lit

        # Locations visited at least once (for insight on first entry)
        self.visited_locations = set()   # set of location ids

        # For optional logging of snapshots
        self.state_history = []

        # Treat the starting location as a first-time visit and grant Insight
        self._handle_first_time_location_entry(self.curr_location)

    # ---------- Location entry helper ----------
    def _handle_first_time_location_entry(self, location):
        """Mark a location as entered and, if it's the first time ever,
        grant +1 Insight.
        """
        loc_id = getattr(location, 'id', location.name)
        first_time = loc_id not in self.visited_locations

        # Mark visited
        self.visited_locations.add(loc_id)
        location.has_been_visited = True

        if first_time:
            print_game_output(f"You glimpse new horrors in {location.name}.")
            # gain_insight already calls log_state
            self.gain_insight(1)
        else:
            # Just record a state snapshot on re-entry
            self.log_state()

    # ---------- Movement and description ----------
    def describe(self):
        print_game_output(f"You are at: {self.curr_location.name}")
        print_game_output(self.curr_location.description)
        self.describe_exits()
        self.describe_items_here()
        self.describe_characters_here()
        self.describe_status()

    def describe_exits(self):
        exits = [d.capitalize() for d in self.curr_location.connections.keys()]
        if exits:
            print_game_output("Exits: " + ", ".join(exits))
        else:
            print_game_output("There are no obvious exits.")

    def describe_items_here(self):
        if not self.curr_location.items:
            print_game_output("You see nothing of note here.")
            return
        print_game_output("You see:")
        for name, item in self.curr_location.items.items():
            count = self.curr_location.item_counts[name]
            if count > 1:
                plural_name = name if name.endswith('s') else name + 's'
                print_game_output(f"  - {plural_name} (x{count}): {item.description}")
            else:
                print_game_output(f"  - {name}: {item.description}")

    def describe_characters_here(self):
        chars = self.curr_location.get_characters_summary()
        if not chars:
            return
        print_game_output("Present here:")
        for char_id, count in chars.items():
            label = f"{char_id} (x{count})" if count > 1 else char_id
            print_game_output(f"  - {label}")

    def describe_inventory(self):
        if not self.inventory:
            print_game_output("You are carrying nothing.")
            return
        print_game_output("You are carrying:")
        for name, item in self.inventory.items():
            count = self.inventory_counts[name]
            if count > 1:
                plural_name = name if name.endswith('s') else name + 's'
                print_game_output(f"  - {plural_name} (x{count})")
            else:
                print_game_output(f"  - {name}")

    def describe_status(self):
        print_game_output(
            f"Health: {self.health}/{self.max_health}  |  "
            f"Weapon +{self.weapon_level}  |  "
            f"Insight: {self.insight}/{self.max_insight}  "
            f"(Area Insight: {self.curr_location.insight_level})"
        )
        print_game_output(
            f"Blood Echoes: {self.blood_echoes}  |  Lamps unlocked: {self.lamps_unlocked_count}"
        )

    @property
    def lamps_unlocked_count(self):
        return len(self.unlocked_lamp_locations)

    def go(self, direction):
        direction = direction.lower()
        if direction not in self.curr_location.connections:
            print_game_output("You can't go that way.")
            return
        if self.curr_location.is_blocked(direction, self):
            return

        new_loc = self.curr_location.connections[direction]
        travel_desc = self.curr_location.travel_descriptions.get(direction, "")
        if travel_desc:
            print_game_output(travel_desc)
        self.curr_location = new_loc

        # Handle first-time visit insight & logging
        self._handle_first_time_location_entry(self.curr_location)

        # Then describe the new area
        self.describe()

    # ---------- Inventory management ----------
    def is_in_inventory(self, item_name):
        return self.inventory_counts[item_name] > 0

    def add_to_inventory(self, item):
        # Keep a canonical Item per name, but track count separately.
        if item.name not in self.inventory:
            self.inventory[item.name] = item
        self.inventory_counts[item.name] += 1
        count = self.inventory_counts[item.name]
        if count > 1:
            plural_name = item.name if item.name.endswith('s') else item.name + 's'
            print_game_output(f"You now have {plural_name} (x{count}).")
        else:
            print_game_output(f"You take the {item.name}.")
        self.log_state()

    def consume_inventory_item(self, item_name, amount=1):
        """Consume `amount` copies of an item from inventory.
        Returns True if at least one was consumed.
        """
        if self.inventory_counts[item_name] <= 0:
            return False
        self.inventory_counts[item_name] -= amount
        if self.inventory_counts[item_name] <= 0:
            # Remove canonical item entry completely.
            self.inventory_counts[item_name] = 0
            self.inventory.pop(item_name, None)
        self.log_state()
        return True

    def remove_from_inventory(self, item_name):
        """Drop a single copy of an item into the current location."""
        if self.inventory_counts[item_name] <= 0:
            print_game_output("You don't have that.")
            return None

        item = self.inventory.get(item_name)
        if item is None:
            print_game_output("You don't have that.")
            return None

        # Decrease count, potentially removing the canonical entry.
        self.inventory_counts[item_name] -= 1
        if self.inventory_counts[item_name] <= 0:
            self.inventory_counts[item_name] = 0
            self.inventory.pop(item_name, None)

        self.curr_location.add_item(item, count=1)
        print_game_output(f"You drop a {item_name}.")
        self.log_state()
        return item

    def take(self, item_name):
        item_name = item_name.lower()
        if item_name not in self.curr_location.items:
            print_game_output("There is no such item here.")
            return
        item = self.curr_location.remove_item(item_name, count=1)
        if item:
            self.add_to_inventory(item)

    # ---------- Combat-ish helpers ----------
    def take_damage(self, amount):
        self.health = max(0, self.health - amount)
        print_game_output(f"You take {amount} damage! Health is now {self.health}.")
        self.log_state()

    def heal(self, amount):
        old = self.health
        self.health = min(self.max_health, self.health + amount)
        restored = self.health - old
        print_game_output(f"You recover {restored} health. Health is now {self.health}.")
        self.log_state()

    def upgrade_weapon(self, levels=1):
        self.weapon_level += levels
        print_game_output(f"Your weapon has been strengthened to +{self.weapon_level}.")
        self.log_state()

    # ---------- Blood Echoes ----------
    def gain_blood_echoes(self, amount):
        if amount <= 0:
            return
        self.blood_echoes += amount
        print_game_output(f"You gain {amount} Blood Echoes. Total: {self.blood_echoes}.")
        self.log_state()

    def spend_blood_echoes(self, amount) -> bool:
        if amount <= 0:
            return True
        if self.blood_echoes < amount:
            print_game_output("You lack sufficient Blood Echoes.")
            return False
        self.blood_echoes -= amount
        print_game_output(f"You spend {amount} Blood Echoes. Remaining: {self.blood_echoes}.")
        self.log_state()
        return True

    # ---------- Insight management ----------
    def gain_insight(self, amount=1):
        old = self.insight
        self.insight = min(self.max_insight, self.insight + amount)
        gained = self.insight - old
        if gained > 0:
            print_game_output(f"You gain {gained} Insight. Insight is now {self.insight}.")
        else:
            print_game_output("Your mind cannot grasp any more Insight.")
        self.log_state()

    def lose_insight(self, amount=1):
        old = self.insight
        self.insight = max(0, self.insight - amount)
        lost = old - self.insight
        if lost > 0:
            print_game_output(f"You lose {lost} Insight. Insight is now {self.insight}.")
        else:
            print_game_output("Your Insight cannot sink any lower.")
        self.log_state()

    # ---------- Lamps ----------
    def unlock_lamp_here(self):
        """Mark the current location as having an unlocked lamp.
        This only increments the count once per location.
        """
        loc_id = self.curr_location.id
        if loc_id in self.unlocked_lamp_locations:
            print_game_output("This lamp is already lit.")
            return
        self.unlocked_lamp_locations.add(loc_id)
        print_game_output(
            f"You light the lamp at {self.curr_location.name}. Lamps unlocked: {self.lamps_unlocked_count}."
        )
        self.log_state()

    # ---------- State snapshot ----------
    def get_state(self):
        """Return a dictionary capturing the current game state."""
        # Build stacked inventory strings
        inventory_display = []
        for name in self.inventory.keys():
            count = self.inventory_counts[name]
            if count > 1:
                inventory_display.append(f"{name} (x{count})")
            else:
                inventory_display.append(name)

        # Location items summary
        loc_item_summary = {}
        for name, itm in self.curr_location.items.items():
            loc_item_summary[name] = self.curr_location.item_counts[name]

        # Location characters summary
        loc_char_summary = self.curr_location.get_characters_summary()

        return {
            'location_id': self.curr_location.id,
            'location': self.curr_location.name,
            'health': self.health,
            'max_health': self.max_health,
            'weapon_level': self.weapon_level,
            'inventory': inventory_display,
            'location_items': loc_item_summary,
            'location_characters': loc_char_summary,
            'insight': self.insight,
            'max_insight': self.max_insight,
            'location_insight': self.curr_location.insight_level,
            'blood_echoes': self.blood_echoes,
            'lamps_unlocked': self.lamps_unlocked_count,
            'visited_locations': list(self.visited_locations),
        }

    def log_state(self):
        """Append a snapshot of the current state to `state_history`."""
        self.state_history.append(self.get_state())



## Dataset Loading: Locations, Items, and Linear Connections

The helpers below build the world from the .json dataset.

In [4]:

def _load_item_definitions(dataset_root="CMSC-491-Project-dataset"):
    """Load all Item definitions from the dataset.

    Returns a dict: item_id -> Item instance.
    """
    item_defs = {}

    items_path = os.path.join(dataset_root, "Data", "Items")
    if os.path.isdir(items_path):
        # Unzipped case
        for fname in os.listdir(items_path):
            if not fname.endswith(".json"):
                continue
            full = os.path.join(items_path, fname)
            with open(full, "r", encoding="utf-8") as f:
                data = json.load(f)
            item_id = data.get("id") or os.path.splitext(fname)[0]
            # Prefer alias, then name, then fallback from id
            display_name = data.get("alias") or data.get("name") or item_id
            desc = data.get("description", "")
            itm = Item(item_id=item_id, name=display_name.lower(), description=desc)
            item_defs[item_id] = itm
    else:
        # Try zipped case
        zip_name = dataset_root + ".zip"
        if not os.path.isfile(zip_name):
            raise FileNotFoundError(f"Could not find items directory or zip at {zip_name}")
        with zipfile.ZipFile(zip_name, 'r') as z:
            prefix = dataset_root + "/Data/Items/"
            for name in z.namelist():
                if not name.startswith(prefix) or not name.endswith('.json'):
                    continue
                with z.open(name) as f:
                    data = json.load(f)
                item_id = data.get("id") or os.path.splitext(os.path.basename(name))[0]
                display_name = data.get("alias") or data.get("name") or item_id
                desc = data.get("description", "")
                itm = Item(item_id=item_id, name=display_name.lower(), description=desc)
                item_defs[item_id] = itm

    return item_defs


def build_world_from_dataset(dataset_root="CMSC-491-Project-dataset"):
    """Build locations and connections from the dataset.

    Returns `(start_location, locations_by_id)`.
    The chain is treated as linear using the `previous` and `next` fields,
    mapping `next` to `north` and `previous` to `south`.
    """
    # ---- Load items first ----
    item_defs = _load_item_definitions(dataset_root)

    # ---- Load location JSONs ----
    locations_by_id = {}
    loc_records = {}

    loc_path = os.path.join(dataset_root, "Data", "Locations")
    if os.path.isdir(loc_path):
        # Unzipped
        for fname in os.listdir(loc_path):
            if not fname.endswith(".json"):
                continue
            full = os.path.join(loc_path, fname)
            with open(full, "r", encoding="utf-8") as f:
                data = json.load(f)
            loc_id = data["id"]
            loc_records[loc_id] = data
    else:
        # Zipped
        zip_name = dataset_root + ".zip"
        if not os.path.isfile(zip_name):
            raise FileNotFoundError(f"Could not find locations directory or zip at {zip_name}")
        with zipfile.ZipFile(zip_name, 'r') as z:
            prefix = dataset_root + "/Data/Locations/"
            for name in z.namelist():
                if not name.startswith(prefix) or not name.endswith('.json'):
                    continue
                with z.open(name) as f:
                    data = json.load(f)
                loc_id = data["id"]
                loc_records[loc_id] = data

    if not loc_records:
        raise RuntimeError("No locations found in dataset.")

    # ---- Create Location objects ----
    for loc_id, data in loc_records.items():
        name = data.get("name", loc_id)
        desc = data.get("description", "")
        # Default insight 0 for now; can be tuned per area
        loc = Location(loc_id=loc_id, name=name, description=desc, insight_level=0)

        # Characters
        for char in data.get("characters", []):
            char_id = char.get("id")
            count = int(char.get("count", 1))
            if char_id:
                loc.add_character(char_id, count=count)

        # Items
        for entry in data.get("items", []):
            item_id = entry.get("item_id")
            count = int(entry.get("count", 1))
            if not item_id or item_id not in item_defs:
                continue
            itm_template = item_defs[item_id]
            loc.add_item(itm_template, count=count)

        locations_by_id[loc_id] = loc

    # ---- Hook up linear connections via previous/next ----
    start_location = None
    for loc_id, data in loc_records.items():
        loc = locations_by_id[loc_id]
        prev_id = data.get("previous")
        next_id = data.get("next")

        if prev_id and prev_id in locations_by_id:
            loc.add_connection('south', locations_by_id[prev_id])
        if next_id and next_id in locations_by_id:
            loc.add_connection('north', locations_by_id[next_id])

    # Heuristic: starting location is any location that has no `previous` field
    # or where `previous` is empty / None.
    for loc_id, data in loc_records.items():
        prev_id = data.get("previous")
        if not prev_id:
            start_location = locations_by_id[loc_id]
            break

    if start_location is None:
        # Fallback: arbitrary first location
        start_location = next(iter(locations_by_id.values()))

    return start_location, locations_by_id



## Initialize Game Test


In [5]:

try:
    start_loc, locs_by_id = build_world_from_dataset("CMSC-491-Project-dataset")
    game = BloodborneGame(
        start_at=start_loc,
        all_locations=locs_by_id,
        max_health=100,
        weapon_level=1,
        starting_insight=0,
        starting_blood_echoes=0,
    )
    # Light starting lamp
    game.unlock_lamp_here()
    game.describe()
    print("\nInitial game state snapshot:")
    print(game.get_state())
except Exception as e:
    print("Error building world from dataset:", e)
    print("Make sure the dataset folder or zip is in the same directory as this notebook.")


You glimpse new horrors in Mob Plaza.
You gain 1 Insight. Insight is now 1.
You light the lamp at Mob Plaza. Lamps unlocked: 1.
You are at: Mob Plaza
A crowd of frenzied townsfolk surround a burning effigy, chanting and hunting anything that moves.
There are no obvious exits.
You see nothing of note here.
Present here:
  - huntsman (x6)
  - rabid_dog (x2)
Health: 100/100  |  Weapon +1  |  Insight: 1/99  (Area Insight: 0)
Blood Echoes: 0  |  Lamps unlocked: 1

Initial game state snapshot:
{'location_id': 'loc_mob_plaza', 'location': 'Mob Plaza', 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': [], 'location_items': {}, 'location_characters': {'huntsman': 6, 'rabid_dog': 2}, 'insight': 1, 'max_insight': 99, 'location_insight': 0, 'blood_echoes': 0, 'lamps_unlocked': 1, 'visited_locations': ['loc_mob_plaza']}
